In [1]:
import json
from portfolio import Portfolio
from optimizer import PortfolioOptimizer
from simulations import MonteCarloSimulation
from utils import (
    get_simulation_insights,
    display_optimal_weights
)

#Snowpark lib
from snowflake.snowpark import Session
import pandas as pd
from fosforml.model_manager.snowflakesession import get_session
my_session = get_session()

config = {
    "TICKERS" : [], "START_DATE" : '2020-01-01', "END_DATE" : '2023-01-01', "INITIAL_INVESTMENT" : 100000,
    "NUM_SIMULATIONS" : 10000, "TIME_HORIZON" : 252, "RISK_FREE_RATE" : 0.02, "WEIGHTS" : None, "OPTIMIZE" : True, "BALANCED" : False
}

In [2]:
table_name = 'STOCKS_DATA'
sf_df = my_session.sql("select * from {}".format(table_name))
df = sf_df.to_pandas()

In [19]:
tickers = ["MTFS", "ALPA", "TCA", "ABC"]
config["TICKERS"] = tickers

In [20]:
def load_data(df, tickers: list):
    stock_data = {}
    min_len = []
    for ticker in tickers:
        data = df[df["STOCK"] == ticker]
        stock_data[ticker] = list(data["ADJ_CLOSE"])
        min_len.append(len(data))
    last_len = min(min_len)
    for ticker in tickers:
        stock_data[ticker] = stock_data[ticker][:last_len]
    
    stock_data = pd.DataFrame(stock_data)
    return stock_data

# load_data(df, config["TICKERS"]).head()

In [22]:
def main(config=config):
    
    # Extract configuration parameters
    tickers = config.get('TICKERS', ["MTFS", "ALPA", "OOGGL"])
    start_date = config.get('START_DATE')
    end_date = config.get('END_DATE')
    initial_investment = config.get('INITIAL_INVESTMENT', 1000)
    num_simulations = config.get('NUM_SIMULATIONS', 10000)
    time_horizon = config.get('TIME_HORIZON', 252)
    risk_free_rate = config.get('RISK_FREE_RATE', 0.0)
    custom_weights = config.get('WEIGHTS')
    optimization_config = config.get('optimization', {})
    optimize = optimization_config.get('OPTIMIZE', True)
    balanced = optimization_config.get('BALANCED', False)
    # Load data
    stock_data = load_data(df, tickers)
    
    # Create portfolio
    portfolio = Portfolio(stock_data)
    portfolio.calculate_returns()
    
    # Annualize returns and covariance
    expected_returns = portfolio.returns.mean() * 252
    covariance_matrix = portfolio.returns.cov() * 252

    # return covariance_matrix, expected_returns
    
    # Determine weights
    if optimize:
        optimizer = PortfolioOptimizer(
            expected_returns,
            covariance_matrix,
            risk_free_rate=risk_free_rate
        )
        if balanced:
            optimal_weights = optimizer.minimize_volatility(target_return=expected_returns.mean())
            print("\nOptimal Balanced Portfolio Weights:")
        else:
            optimal_weights = optimizer.maximize_sharpe_ratio()
            print("\nOptimal Portfolio Weights to Maximize Sharpe Ratio:")
        display_optimal_weights(stock_data.columns, optimal_weights)
        weights = optimal_weights
    elif custom_weights:
        weights = custom_weights
        print("\nUsing Custom Weights:")
        display_optimal_weights(stock_data.columns, weights)
    else:
        num_assets = len(expected_returns)
        weights = [1.0 / num_assets] * num_assets
        print("\nUsing Equal Weights:")
        display_optimal_weights(stock_data.columns, weights)

    print(weights)
    # Perform Monte Carlo Simulation
    simulation = MonteCarloSimulation(portfolio.returns, initial_investment, weights)
    all_cumulative_returns, final_portfolio_values = simulation.run_simulation(
        num_simulations, time_horizon
    )
    
    # Analyze Results
    final_insights = get_simulation_insights(final_portfolio_values, initial_investment)
    print("**************************")
    for k, v in final_insights.items():
        print(k,":", v)
    # print(final_insights)
    return all_cumulative_returns, final_portfolio_values
    
    # Plot results
    # plot_simulation_results(all_cumulative_returns, final_portfolio_values)



a, b = main()


Optimal Portfolio Weights to Maximize Sharpe Ratio:
  Ticker  Weight
0   MTFS  0.3105
1   ALPA  0.5265
2    TCA  0.1630
3    ABC  0.0000
[0.3105002  0.52651881 0.16298099 0.        ]
**************************
Initial Investment : $100,000.00
Expected Final Portfolio Value : $128,303.15
Median Final Portfolio Value : $125,166.54
Standard Deviation of Final Portfolio Value : $29,646.14
Value at Risk (VaR 95%) : $13,728.36
Conditional Value at Risk (CVaR 95%) : $21,138.27
Probability of Loss : 16.60%
Sharpe Ratio : 0.9547


In [26]:
tickers = config.get('TICKERS', ["MTFS", "ALPA", "OOGGL"])
start_date = config.get('START_DATE')
end_date = config.get('END_DATE')
initial_investment = config.get('INITIAL_INVESTMENT', 1000)
num_simulations = config.get('NUM_SIMULATIONS', 10000)
time_horizon = config.get('TIME_HORIZON', 252)
risk_free_rate = config.get('RISK_FREE_RATE', 0.0)
custom_weights = config.get('WEIGHTS')
optimization_config = config.get('optimization', {})
optimize = optimization_config.get('OPTIMIZE', True)
balanced = optimization_config.get('BALANCED', False)
# Load data
stock_data = load_data(df, tickers)

# Create portfolio
portfolio = Portfolio(stock_data)
portfolio.calculate_returns()
weights = [0.25, 0.25, 0.25, 0.25]
simulation = MonteCarloSimulation(portfolio.returns, initial_investment, weights)
all_cumulative_returns, final_portfolio_values = simulation.run_simulation(
    num_simulations, time_horizon
)

# Analyze Results
final_insights = get_simulation_insights(final_portfolio_values, initial_investment)
print("**************************")
for k, v in final_insights.items():
    print(k,":", v)
# print(final_insights)

**************************
Initial Investment : $100,000.00
Expected Final Portfolio Value : $123,068.71
Median Final Portfolio Value : $119,671.71
Standard Deviation of Final Portfolio Value : $28,199.69
Value at Risk (VaR 95%) : $16,797.71
Conditional Value at Risk (CVaR 95%) : $23,620.73
Probability of Loss : 21.18%
Sharpe Ratio : 0.8180


In [27]:
table_name = 'MEMBER_MASTER'
users_df = my_session.sql("select * from {}".format(table_name))
users = users_df.to_pandas()

In [31]:
stocks = list(df["STOCK"].unique())

import random
portfolio_stocks = [2,3,3,4,4,5,5,6,6,7,8,9]

portfolio = [random.sample(stocks, random.choice(portfolio_stocks))for i in range(users.shape[0])]

In [34]:
users["PORTFOLIO"] = portfolio

In [35]:
users["INTITIAL_INVVESTMENT"] = users["MEMBER_ID"].apply(lambda x: random.choice([1000, 10000, 5000, 50000, 1000000, 80000, 90000, 11000, 45000,
                                                                                  55000, 1000000, 40000, 88000, 97000, 49000, 69000])

,MEMBER_ID,MEMBER_NAME,MEMBER_DOB,MEMBER_EMPLOYMENT,MEMBER_GENDER,MEMBER_CITY_TOWN,MEMBER_STATE,MEMBER_CONTACT_VERIFIED,PORTFOLIO
0,MID000001,Vallie Bachman,08-01-1980,The University of Sydney,Female,Hobart,Tasmania,N,"[TWM, NTIC, ALPA]"
1,MID000002,Jade Strassner,19-11-1986,Curtin University,Female,Gladstone,Queensland,N,"[SM, TWM, ALPA, ABT, JJN, LTSA, GMNA, KRM]"
2,MID000003,Mari Candlish,27-01-1989,University of New England,Female,Walgett,New South Wales,Y,"[FWC, GMNA, ALPA, ABC]"
3,MID000004,Serina Leto,08-03-1984,Griffith University,Female,Paynesville,Victoria,Y,"[GS, ABT, KRM, ABC, ALPA, LTSA, PG, NEK, TWM]"
4,MID000005,Brett Butzen,23-01-1970,La Trobe University,Male,Palmerston,Northern Territory,Y,"[OOGGL, JJN, ABT, TWM, KRM]"
